In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session with Iceberg configurations
spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .getOrCreate()

# Verify Spark session creation
spark.sql("SHOW DATABASES").show()

25/03/20 17:06:02 WARN Utils: Your hostname, codespaces-785606 resolves to a loopback address: 127.0.0.1; using 10.0.0.44 instead (on interface eth0)
25/03/20 17:06:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/workspaces/learn_iceberg/iceberg_pyspark_project/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9692d9fb-5240-445e-8c4f-95ea04be2e01;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.5_2.12/1.5.2/iceberg-spark-runtime-3.5_2.12-1.5.2.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2!iceberg-spark-runtime-3.5_2.12.jar (4359ms)
:: resolution report :: resolve 1806ms :: artifacts dl 4365ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

+---------+
|namespace|
+---------+
|  default|
+---------+



25/03/20 17:06:24 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
# Create an Iceberg table
spark.sql("""
  CREATE TABLE local.schema.users (
    id INT,
    name STRING,
    age INT
  ) USING iceberg""")

# Insert some sample data
spark.sql("""
  INSERT INTO local.schema.users VALUES
    (1, 'Alice', 30),
    (2, 'Bob', 25),
    (3, 'Charlie', 35)""")

# Query the data
result = spark.sql("SELECT * FROM local.schema.users")
result.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 30|
|  2|    Bob| 25|
|  3|Charlie| 35|
+---+-------+---+



In [4]:
result = spark.sql("SELECT * FROM local.schema.users")
result.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 30|
|  2|pradeep| 25|
|  3|Charlie| 35|
+---+-------+---+



In [3]:
spark.sql("update local.schema.users set name = 'pradeep' where id =2")

DataFrame[]

In [7]:
# Try to delete record
spark.sql("delete from local.schema.users where id = 2")

DataFrame[]

In [13]:
spark.sql("SELECT * FROM local.schema.users").show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 30|
|  2|    Bob| 25|
|  3|Charlie| 35|
+---+-------+---+



In [9]:
# Create new table as target.
spark.sql("create table local.schema.users_tgt as SELECT * FROM local.schema.users")
spark.sql("SELECT * FROM local.schema.users_tgt").show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  3|Charlie| 35|
|  1|  Alice| 30|
+---+-------+---+



In [12]:
spark.sql("""
  INSERT INTO local.schema.users VALUES
    (2, 'Bob', 25)""")

DataFrame[]

In [14]:
# Merge _tgt table with users table
spark.sql("MERGE INTO local.schema.users_tgt AS target USING local.schema.users AS source ON target.id = source.id WHEN NOT MATCHED THEN INSERT *")
spark.sql("SELECT * FROM local.schema.users_tgt").show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  2|    Bob| 25|
|  3|Charlie| 35|
|  1|  Alice| 30|
+---+-------+---+

